In [1]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import BertTokenizer,BertModel,BertTokenizerFast,DebertaTokenizer,DebertaTokenizerFast, AutoTokenizer
from utils.MIND import MIND
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [ ]:
t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = DebertaTokenizerFast.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
# config.impr_size = 100
# config.mode = 'dev'
# config.scale = 'large'

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
# manager.gather_same_user_impr()
a = MIND(manager)

[2021-10-07 20:30:29,760] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDdemo_train/10/behaviors.pkl
[2021-10-07 20:30:29,773] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDdemo_train/news.pkl
[2021-10-07 20:30:30,762] INFO (utils.utils) deduplicating...


In [8]:
sampler1 = Partition_Sampler(a, 3, 0)
sampler2 = Partition_Sampler(a, 3, 1)
sampler3 = Partition_Sampler(a, 3, 2)

In [10]:
list(sampler1)[0],list(sampler1)[-1],list(sampler2)[0],list(sampler2)[-1],list(sampler3)[0],list(sampler3)[-1]

(0, 982, 983, 1965, 1966, 2949)

In [5]:
sampler1.start, sampler1.end, sampler3.start, sampler3.end, sampler1.partition_points

(0, 983, 1966, 2950, [0, 983, 1966, 2949])

In [ ]:
from utils.MIND import MIND_news, MIND_history
b = MIND_news(manager)
c = MIND_history(manager)

In [ ]:
c[1]

In [2]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDlarge_test/news.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

In [3]:
ids.shape

(120962, 512)

In [ ]:
a = c

news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k,z in zip(news,mask,subword,dedup):
    print(i,j,k,z)

In [7]:
[[0,0]]*10

[[0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0]]

In [14]:
i = news['encoded_news']

100

In [1]:
partition_points = range(0, 289, 10)

In [3]:
[x for x in partition_points]

[0,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280]